In [1]:
import sys
sys.path.append('C:/Users/13447/Documents/GitHub/master_thesis')

In [21]:

# import pysparnn.cluster_index as ci
# import scipy.sparse
# import os
import math
import numpy as np
import pandas as pd
import random
import scipy.sparse as sp
from utils import train_test_split, df_to_matrix ,matrix_to_df_2, threshold_interactions_df, matrix_to_df,set_intersection,get_0_and_p_index,set_diff, matrix_to_full_df, threshold_interactions_df_plus, train_test_split_csr

!pip install surprise
from surprise import Reader, accuracy
from surprise import SVD
from surprise import Dataset
from collections import Counter
from collections import defaultdict

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from pct.tree.heuristic.Heuristic import Heuristic5
from pct.tree.heuristic.NumericHeuristic_pair import NumericHeuristic5
from pct.tree.splitter.splitterCopy2 import Splitter
from pct.tree.tree import Tree

In [4]:

my_seed = 7
random.seed(my_seed)
np.random.seed(my_seed)

def load_user_item_matrix(filepath):
    data = []

    with open(filepath, 'r') as f:
        while True:
            user_line = f.readline()
            if not user_line:
                break

            user_line = user_line.strip()
            if '|' not in user_line:
                continue

            user_id, num_ratings = user_line.split('|')
            user_id = int(user_id)
            num_ratings = int(num_ratings)

            for _ in range(num_ratings):
                item_line = f.readline().strip()
                parts = item_line.split('\t')
                if len(parts) < 2:
                    continue
                item_id, score = parts[0], parts[1]
                data.append([user_id, int(item_id), int(score)])

    df = pd.DataFrame(data, columns=['user_id', 'item_id', 'rating'])
    return df


In [5]:
def safe_int(x):
    return int(x) if x != 'None' else 0


def load_track_data(filepath):
    track_data = []
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split('|')
            track_id = safe_int(parts[0])
            album_id = safe_int(parts[1])
            artist_id = safe_int(parts[2])
            genre_ids = [safe_int(g) for g in parts[3:]] if len(parts) > 3 else []
            track_data.append([track_id, album_id, artist_id, genre_ids])
    return pd.DataFrame(track_data, columns=['track_id', 'album_id', 'artist_id', 'genre_ids'])

def load_album_data(filepath):
    album_data = []
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split('|')
            album_id = safe_int(parts[0])
            artist_id = safe_int(parts[1])
            genre_ids = [safe_int(g) for g in parts[2:]] if len(parts) > 2 else []
            album_data.append([album_id, artist_id, genre_ids])
    return pd.DataFrame(album_data, columns=['album_id', 'artist_id', 'genre_ids'])


# read artistData2
def load_artist_data(filepath):
    with open(filepath, 'r') as f:
        artist_ids = [int(line.strip()) for line in f]
    return pd.DataFrame(artist_ids, columns=['artist_id'])

# read genreData2
def load_genre_data(filepath):
    with open(filepath, 'r') as f:
        genre_ids = [int(line.strip()) for line in f]
    return pd.DataFrame(genre_ids, columns=['genre_id'])

In [6]:
track_df = load_track_data('D:/dataset/ydata-ymusic-kddcup-2011-track2/ydata-ymusic-kddcup-2011-track2/trackData2.txt')
track_df

,track_id,album_id,artist_id,genre_ids
0,1,106710,281667,"[214765, 162234, 155788]"
1,2,280977,233685,"[131552, 173467, 48505]"
2,3,38422,219136,"[61215, 201738, 88853]"
3,4,119529,166863,"[17453, 35389]"
4,5,16742,294690,"[61215, 34486, 274088]"
...,...,...,...,...
224036,296100,166516,33011,"[274088, 199606, 88853]"
224037,296101,0,0,[]
224038,296102,153644,289056,"[158282, 139095, 242383]"
224039,296105,68336,6613,[82064]


In [11]:
album_df = load_album_data('D:/dataset/ydata-ymusic-kddcup-2011-track2/ydata-ymusic-kddcup-2011-track2/albumData2.txt')
album_df

,album_id,artist_id,genre_ids
0,0,0,[214765]
1,6,228091,"[158282, 81520, 242383]"
2,19,85028,[103715]
3,30,16832,[31567]
4,33,26330,"[149962, 209270]"
...,...,...,...
52824,296104,125866,[158282]
52825,296106,221637,"[116130, 9432]"
52826,296107,0,[61215]
52827,296108,93277,[61215]


In [13]:
artist_df = load_artist_data('D:/dataset/ydata-ymusic-kddcup-2011-track2/ydata-ymusic-kddcup-2011-track2/artistData2.txt')
artist_df

,artist_id
0,16
1,23
2,35
3,40
4,49
...,...
18669,295987
18670,296007
18671,296012
18672,296013


In [15]:
genre_df = load_genre_data('D:/dataset/ydata-ymusic-kddcup-2011-track2/ydata-ymusic-kddcup-2011-track2/genreData2.txt')
genre_df

,genre_id
0,208
1,315
2,642
3,1075
4,1271
...,...
562,292093
563,293670
564,293688
565,294138


In [17]:
train_path = 'D:/dataset/ydata-ymusic-kddcup-2011-track2/ydata-ymusic-kddcup-2011-track2/trainIdx2.txt'
my_seed = 7
random.seed(my_seed)
np.random.seed(my_seed)
train_df = load_user_item_matrix(train_path)


In [18]:
train_df

,user_id,item_id,rating
0,0,28341,90
1,0,51210,90
2,0,79500,90
3,0,82317,90
4,0,98399,90
...,...,...,...
61944401,249011,270557,90
61944402,249011,273574,90
61944403,249011,286938,90
61944404,249011,287681,80


In [10]:
duplicate_counts = train_df.groupby(['user_id', 'item_id']).size()
duplicates = duplicate_counts[duplicate_counts > 1]

print(f"🔁 Number of duplicated (user_id, item_id) pairs: {len(duplicates)}")
print(duplicates.head())


🔁 Number of duplicated (user_id, item_id) pairs: 0
Series([], dtype: int64)


In [68]:
print(train_df['item_id'].max())      # highest item_id
print(track_df['track_id'].max())     # highest track_id
print(train_df['item_id'].isin(track_df['track_id']).mean())  # proportion to match

print(train_df['item_id'].isin(album_df['album_id']).mean())

print(train_df['item_id'].isin(artist_df['artist_id']).mean())

print(train_df['item_id'].isin(genre_df['genre_id']).mean())



296110
296110
0.4385845107627636
0.1925648621120041
0.3114063600835885
0.05744426704164376


In [23]:
def build_item_metadata(track_df, album_df, artist_df, genre_df):
    rows = []

    # 1. Tracks
    for _, row in track_df.iterrows():
        rows.append({
            'item_id': row['track_id'],
            'item_type': 'track',
            'artist_id': row['artist_id'],
            'genre_ids': row['genre_ids']
        })

    # 2. Albums
    for _, row in album_df.iterrows():
        rows.append({
            'item_id': row['album_id'],
            'item_type': 'album',
            'artist_id': row['artist_id'],
            'genre_ids': row['genre_ids']
        })

    # 3. Artists
    for artist_id in artist_df['artist_id']:
        rows.append({
            'item_id': artist_id,
            'item_type': 'artist',
            'artist_id': artist_id,
            'genre_ids': 0
        })

    # 4. Genres
    for genre_id in genre_df['genre_id']:
        rows.append({
            'item_id': genre_id,
            'item_type': 'genre',
            'artist_id': 0,
            'genre_ids': [genre_id]
        })

    return pd.DataFrame(rows)

In [25]:
item_meta_df = build_item_metadata(track_df, album_df, artist_df, genre_df)

train_with_meta = train_df.merge(item_meta_df, on='item_id', how='inner')

# When loading the data, sort by a stable column (e.g., user_id)
train_with_meta = train_with_meta.sort_values('user_id').reset_index(drop=True)

train_with_meta

,user_id,item_id,rating,item_type,artist_id,genre_ids
0,0,28341,90,artist,28341,0
1,0,161823,30,artist,161823,0
2,0,79500,90,artist,79500,0
3,0,182018,70,artist,182018,0
4,0,180487,90,artist,180487,0
...,...,...,...,...,...,...
61944401,249011,131552,80,genre,0,[131552]
61944402,249011,286938,90,artist,286938,0
61944403,249011,287681,80,genre,0,[287681]
61944404,249011,252215,80,genre,0,[252215]


In [27]:
df = train_with_meta.copy()

# Filter: keep only artist & genre (drop album, track)
df = df[~df['item_type'].isin(['album', 'track'])].copy()

print(f"🎯 Sample shape: {df.shape}")
print(f"👤 Users: {df['user_id'].nunique()}, 🎵 Items: {df['item_id'].nunique()}")

🎯 Sample shape: (22848233, 6)
👤 Users: 248344, 🎵 Items: 19241


In [29]:
filtered_df = threshold_interactions_df(df,'user_id','item_id',100,500)

Starting interactions info
Number of rows: 248344
Number of cols: 19241
Sparsity: 0.478%
Ending interactions info
Number of rows: 4644
Number of columns: 8938
Sparsity: 10.674%


In [31]:
# filtered_df = threshold_interactions_df_plus(df, min_items_per_user=500, min_artists_per_user=20, min_genres_per_user=10,min_users_per_item=150)
my_seed = 7
random.seed(my_seed)
np.random.seed(my_seed)
sampled_users = np.random.choice(filtered_df['user_id'].unique(), size=2000, replace=False)
sampled_df = filtered_df[filtered_df['user_id'].isin(sampled_users)]
print(f"🎯 Sample shape: {sampled_df.shape}")
print(f"👤 Users: {sampled_df['user_id'].nunique()}, 🎵 Items: {sampled_df['item_id'].nunique()}")
filtered_df = threshold_interactions_df(sampled_df,'user_id','item_id',100,500)

🎯 Sample shape: (1941009, 6)
👤 Users: 2000, 🎵 Items: 8938
Starting interactions info
Number of rows: 2000
Number of cols: 8938
Sparsity: 10.858%
Ending interactions info
Number of rows: 1701
Number of columns: 5011
Sparsity: 17.890%


In [33]:
all_user_ids = sorted(filtered_df['user_id'].unique().tolist())
filtered_df

,user_id,item_id,rating,item_type,artist_id,genre_ids
1041,9,238709,0,artist,238709,0
1042,9,169510,0,artist,169510,0
1047,9,208084,70,artist,208084,0
1049,9,245398,0,artist,245398,0
1052,9,153166,0,artist,153166,0
...,...,...,...,...,...,...
61933962,248947,83754,30,artist,83754,0
61933963,248947,141799,0,genre,0,[141799]
61933966,248947,141677,0,artist,141677,0
61933967,248947,262458,0,artist,262458,0


In [35]:
def to_semi_binary(r):
    if r > 50:
        return 1
    elif r > 0:
        return 0.1
    else:
        return 0


In [39]:
filtered_df['rating'] = filtered_df['rating'].apply(to_semi_binary)
filtered_df

,user_id,item_id,rating,item_type,artist_id,genre_ids
1041,9,238709,0.0,artist,238709,0
1042,9,169510,0.0,artist,169510,0
1047,9,208084,0.1,artist,208084,0
1049,9,245398,0.0,artist,245398,0
1052,9,153166,0.0,artist,153166,0
...,...,...,...,...,...,...
61933962,248947,83754,0.1,artist,83754,0
61933963,248947,141799,0.0,genre,0,[141799]
61933966,248947,141677,0.0,artist,141677,0
61933967,248947,262458,0.0,artist,262458,0


In [41]:
# item type map
item_type_map = filtered_df.drop_duplicates(subset='item_id')[['item_id', 'item_type']]
item_type_map = dict(zip(item_type_map['item_id'], item_type_map['item_type']))


In [43]:
def get_item_type(item_id):
    return item_type_map.get(item_id, 'unknown')  


In [45]:
print(get_item_type(238709))  
print(get_item_type(141799))  


artist
genre


In [47]:
all_user_ids = sorted(filtered_df['user_id'].unique().tolist())

def split_users_by_ratio(all_user_ids, ratio):
    n = len(all_user_ids)
    split_point = int(n * ratio)
    warm_users = all_user_ids[:split_point]
    cold_users = all_user_ids[split_point:]
    return warm_users, cold_users

# Example ratios from 10% to 50%
# ratios = [0.1, 0.2, 0.3, 0.4, 0.5]
# splits = {r: split_users_by_ratio(all_user_ids, r) for r in ratios}


## 10% warm

In [50]:
warm_users_idx, cold_users_idx = split_users_by_ratio(all_user_ids, 0.1)

df_warm = filtered_df[filtered_df['user_id'].isin(warm_users_idx)].copy()
df_cold = filtered_df[filtered_df['user_id'].isin(cold_users_idx)].copy()

matrix_warm, rid_to_idx_warm, idx_to_rid_warm, cid_to_idx, idx_to_cid = df_to_matrix(
    df_warm, "user_id", "item_id", "rating")


matrix_cold, rid_to_idx_cold, idx_to_rid_cold, _, _ = df_to_matrix( 
    df_cold, "user_id", "item_id", "rating")



In [51]:
df_warm

,user_id,item_id,rating,item_type,artist_id,genre_ids
1041,9,238709,0.0,artist,238709,0
1042,9,169510,0.0,artist,169510,0
1047,9,208084,0.1,artist,208084,0
1049,9,245398,0.0,artist,245398,0
1052,9,153166,0.0,artist,153166,0
...,...,...,...,...,...,...
6037088,24286,253660,0.1,artist,253660,0
6037092,24286,293466,0.1,artist,293466,0
6037095,24286,178994,0.1,artist,178994,0
6037096,24286,219001,0.0,artist,219001,0


In [54]:
df_cold

,user_id,item_id,rating,item_type,artist_id,genre_ids
6058372,24414,267421,0.0,artist,267421,0
6058376,24414,279057,0.0,artist,279057,0
6058388,24414,60300,0.1,artist,60300,0
6058393,24414,121430,0.0,artist,121430,0
6058396,24414,164412,0.1,artist,164412,0
...,...,...,...,...,...,...
61933962,248947,83754,0.1,artist,83754,0
61933963,248947,141799,0.0,genre,0,[141799]
61933966,248947,141677,0.0,artist,141677,0
61933967,248947,262458,0.0,artist,262458,0


In [56]:
def split_and_combine(strategy="artist-only"):
    """Handles both approaches with proper matrix alignment"""
    # Get full cold matrix and mappings
    matrix_cold, rid_to_idx_cold, _, cid_to_idx, _ = df_to_matrix(
        df_cold, "user_id", "item_id", "rating"
    )
    matrix_cold = matrix_cold.tocsr()

    # Create boolean masks
    artist_mask = np.isin(
        np.arange(matrix_cold.shape[1]), 
        [cid_to_idx[iid] for iid in df_cold[df_cold['item_type'] == 'artist']['item_id']]
    )
    genre_mask = ~artist_mask

    # Create aligned matrices
    matrix_cold_artist = matrix_cold.multiply(artist_mask)
    matrix_cold_genre = matrix_cold.multiply(genre_mask)
    matrix_cold_artist = matrix_cold.multiply(artist_mask).tocsr()
    matrix_cold_genre = matrix_cold.multiply(genre_mask).tocsr()

    al_artist, test_cold, _ = train_test_split(
        matrix_cold_artist, 
        split_count=30,
        fraction=None
    )
    
    if strategy == "artist-only":
        X_cold, K_cold, _ = train_test_split_csr(al_artist, 1)  
        return K_cold, X_cold, test_cold
    
    elif strategy == "hybrid":
        X_cold, K_cold, _ = train_test_split_csr(al_artist, 1)
        X_cold = X_cold + matrix_cold_genre
        return K_cold, X_cold, test_cold


In [58]:
my_seed = 7
random.seed(my_seed)
np.random.seed(my_seed)

train_cold_K_artist, X_cold_artist, test_cold_artist  = split_and_combine("artist-only")
train_cold_K_hybrid, X_cold_hybrid, test_cold_hybrid = split_and_combine("hybrid")


In [59]:
# shape of test_cold

print(f"Cold users in X: {len(np.unique(X_cold_artist.nonzero()[0]))}")
print(f"Cold items in X: {len(np.unique(X_cold_artist.nonzero()[1]))}")
print(f"test users in test: {len(np.unique(test_cold_artist.nonzero()[0]))}")
print(f"test items in test: {len(np.unique(test_cold_artist.nonzero()[1]))}")
print(f"train users in train: {len(np.unique(train_cold_K_artist.nonzero()[0]))}")
print(f"train items in train: {len(np.unique(train_cold_K_artist.nonzero()[1]))}")

print("-------------------")

# shape of test_cold
print(f"Shape of test_cold: {test_cold_artist.shape}")
print(f"Shape of train_cold: {train_cold_K_artist.shape}")
print(f"Shape of X_cold: {X_cold_artist.shape}")

Cold users in X: 1531
Cold items in X: 4793
test users in test: 1479
test items in test: 3898
train users in train: 692
train items in train: 574
-------------------
Shape of test_cold: (1531, 5011)
Shape of train_cold: (1531, 5011)
Shape of X_cold: (1531, 5011)


In [62]:
print(f"Cold users in X: {len(np.unique(X_cold_hybrid.nonzero()[0]))}")
print(f"Cold items in X: {len(np.unique(X_cold_hybrid.nonzero()[1]))}")
print(f"test users in test: {len(np.unique(test_cold_hybrid.nonzero()[0]))}")
print(f"test items in test: {len(np.unique(test_cold_hybrid.nonzero()[1]))}")
print(f"train users in train: {len(np.unique(train_cold_K_hybrid.nonzero()[0]))}")
print(f"train items in train: {len(np.unique(train_cold_K_hybrid.nonzero()[1]))}")

print("-------------------")

# shape of test_cold
print(f"Shape of test_cold: {test_cold_hybrid.shape}")
print(f"Shape of train_cold: {train_cold_K_hybrid.shape}")
print(f"Shape of X_cold: {X_cold_hybrid.shape}")

Cold users in X: 1531
Cold items in X: 5011
test users in test: 1475
test items in test: 3980
train users in train: 689
train items in train: 587
-------------------
Shape of test_cold: (1531, 5011)
Shape of train_cold: (1531, 5011)
Shape of X_cold: (1531, 5011)


# X with artist + genre

In [ ]:
print (X_cold_hybrid)

In [66]:
train_df = matrix_to_df_2(train_cold_K_hybrid,idx_to_rid_cold,idx_to_cid)
train_copy = (train_cold_K_hybrid.tolil()).copy()
warm_df = matrix_to_full_df(matrix_warm, idx_to_rid_warm, idx_to_cid)
coldK_df = matrix_to_full_df(train_copy, idx_to_rid_cold, idx_to_cid)
x_df = pd.concat([warm_df, coldK_df], ignore_index=False)

In [68]:
x_df

,238709,169510,208084,245398,153166,257965,31321,79867,185075,35899,...,275477,126760,203794,84071,42783,150884,171378,71326,111572,241362
9,0.0,0.0,0.1,0.0,0.0,0.1,0.1,0.1,0.0,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
217,0.0,0.0,0.0,0.1,0.0,0.1,0.1,0.0,0.0,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
510,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
562,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
248269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
248803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
248908,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
numerical_attributes = x_df.columns.tolist()
categorical_attributes = []  
min_instances = 2  
target_weights = None  

splitter = Splitter(
    min_instances=min_instances,
    numerical_attributes=numerical_attributes,
    categorical_attributes=categorical_attributes,
    target_weights=target_weights
)

Initializing Splitter...


In [72]:
instance_weights = np.ones(len(x_df))


item, error, ranked_items = splitter.find_best_split_item(
    x_df, x_df, instance_weights, return_ranked=True
)

items_ranked = [i for i, _ in ranked_items]
print("Top 20 ranked items:", items_ranked[:20])


Top 20 ranked items: [213777, 224698, 148621, 213902, 232969, 67964, 163783, 93630, 15343, 77961, 237614, 102563, 180478, 288426, 13443, 222610, 126978, 286313, 93288, 64661]


In [74]:
itemA, itemB = splitter.select_pair(items_ranked, x_df, strategy=1)
print(f"Selected pair: itemA={itemA}, itemB={itemB}, type={get_item_type(itemA)}")

Selected pair: itemA=213777, itemB=224698, type=artist


In [76]:
itemA, itemB = splitter.select_pair(items_ranked, x_df, strategy=2)
print(f"Selected pair: itemA={itemA}, itemB={itemB}, type={get_item_type(itemA)}")


[DEBUG] itemA = 213777, vecA = [0 0 0 ... 0 0 0]
  Compared to itemB = 224698, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 148621, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 213902, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 232969, vecB = [0 0 0 ... 0 0 0], similarity = 0.707
  Compared to itemB = 67964, vecB = [0 0 0 ... 0 0 0], similarity = 0.577
  Compared to itemB = 163783, vecB = [0 0 0 ... 0 0 0], similarity = 0.707
  Compared to itemB = 93630, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 15343, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 77961, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 237614, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 102563, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 180478, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 288426, vecB = [0 0 0 ... 0 0 0]

In [78]:
itemA, itemB = splitter.select_pair(items_ranked, x_df, strategy=3)
print(f"Selected pair: itemA={itemA}, itemB={itemB}, type={get_item_type(itemA)}")


[DEBUG] itemA = 213777, vecA = [0 0 0 ... 0 0 0]
  Compared to itemB = 224698, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 148621, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 213902, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 232969, vecB = [0 0 0 ... 0 0 0], similarity = 0.707
  Compared to itemB = 67964, vecB = [0 0 0 ... 0 0 0], similarity = 0.577
  Compared to itemB = 163783, vecB = [0 0 0 ... 0 0 0], similarity = 0.707
  Compared to itemB = 93630, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 15343, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 77961, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 237614, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 102563, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 180478, vecB = [0 0 0 ... 0 0 0], similarity = 1.000
  Compared to itemB = 288426, vecB = [0 0 0 ... 0 0 0]